In [77]:
import os

from dotenv import load_dotenv
from crewai import Crew, Agent, Task, LLM

load_dotenv()

True

In [78]:
serper_api_key = os.getenv("SERPER_API_KEY")

if not serper_api_key:
    raise ValueError("SERPER_API_KEY is missing in environment variables.")

print(f"Loaded SERPER_API_KEY with length {len(serper_api_key)} characters.")


Loaded SERPER_API_KEY with length 40 characters.


In [79]:
ollama_model = os.getenv("OLLAMA_MODEL", "llama3.2:latest")
ollama_url = os.getenv("OLLAMA_URL", "http://localhost:11434")
temperature = float(os.getenv("TEMPERATURE", "0.6"))

if not ollama_model.startswith("ollama/"):
    llm_model = f"ollama/{ollama_model}"
else:
    llm_model = ollama_model

ollamallm = LLM(
    model=llm_model,
    base_url=ollama_url,
    temperature=temperature,
)

print(f"Initialized LLM with model={llm_model} @ {ollama_url}")

Initialized LLM with model=ollama/llama3.2:latest @ http://localhost:11434


In [80]:
test_prompt = "Say hello in one sentence."

response = ollamallm.call(
    messages=[
        {"role": "system", "content": "You are a concise assistant."},
        {"role": "user", "content": test_prompt},
    ]
)

print(response)


Hello.


In [81]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [82]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    allow_delegation=True,
    verbose=True,
    tools=[search_tool, scrape_tool],
    llm=ollamallm
)

In [83]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=ollamallm
)

In [84]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=ollamallm
)






In [85]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=ollamallm
)






In [86]:
report_writer_agent = Agent(
    role="Report Writer",
    goal="Write a comprehensive report summarizing the trading "
         "strategies, market analysis, and risk management "
         "recommendations.",
    backstory="You have 20 years of experience in financial analysis and trading. You are a skilled writer who can synthesize complex information into a clear and concise report. ",
    allow_delegation=True,
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=ollamallm
)



In [87]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [88]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [89]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [90]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [91]:
report_writing_task = Task(
    description=(
        "Write a comprehensive report summarizing the trading "
        "strategies, market analysis, and risk management "
        "recommendations."
    ),
    expected_output=(
        "A comprehensive report merging outputs from all agents and giving comprehensive report with all the details."
    ),
    agent=report_writer_agent,
)

In [92]:
from crewai import Crew, Process

In [93]:
# define the crew with agent and tasks
financial_trading_crew = Crew (
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent,
            report_writer_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task,
           report_writing_task],
    
    verbose=True,
    process=Process.sequential,
)

In [94]:
# specify the input for crew
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}


In [95]:
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3dbc63d1-1d34-4115-a1fd-a5f90cd35494                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling     │
│  and machine learning to identify trends and predict market movements.                                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/macbook/Desktop/crewai_agent_001/.venv/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/Users/macbook/Desktop/crewai_agent_001/.venv/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Analyst                                                                                            │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'Apple earnings report 2025 analysis', 'type': 'search', 'num': 10, 'engine':       │
│  'google'}, 'organic': [{'title': 'Apple reports fourth quarter results', 'link':                               │
│  'https://www.apple.com/newsroom/2025/10/apple-reports-fourth-quarter-results/', 'snippet': 'Apple today        │
│  announced financial results for its fiscal 2025 fourth quarter ended September 27, 2025.', 'position': 1},     │
│  {'title': 'Apple reports third quarter results', 'link':                                                       │
│  'https://www.apple.com/newsroom/2025/07/apple-reports-third-quarter-results/', 'snippet': 'Apple today         │
│  announced financial results for its fiscal 2025 third quarter ended June 28, 2025.', 'position': 2},           │
│  {'title': 'Apple Inc Earnings - Analysis & Highlights for Q3 2025 - AlphaSense', 'link':                       │
│  'https://www.alpha-sense.com/earnings/aapl/', 'snippet': 'EPS: $1.85, a September quarter record. Fiscal year  │
│  2025 revenue: All-time record of $416 billion. Gross margin: 47.2%, up 70 basis points sequentially. ...',     │
│  'position': 3}, {'title': 'Going Into Earnings, Is Apple Stock a Buy, a Sell, or Fairly Valued?', 'link':      │
│  'https://www.morningstar.com/stocks/going-into-earnings-is-apple-stock-buy-sell-or-fairly-valued-9',           │
│  'snippet': "With its 2-star rating, we believe Apple's stock is overvalued compared with our long-term fair    │
│  value estimate of $210 per share, which implies ...", 'position': 4}, {'title': 'Apple reports first quarter   │
│  results', 'link': 'https://www.apple.com/newsroom/2025/01/apple-reports-first-quarter-results/', 'snippet':    │
│  'CUPERTINO, CALIFORNIA Apple today announced financial results for its fiscal 2025 first quarter ended         │
│  December 28, 2024.', 'position': 5, 'sitelinks': [{'title': 'Apple (CA)', 'link':                              │
│  'https://www.apple.com/ca/newsroom/2025/01/apple-reports-first-quarter-results/'}, {'title': 'Apple (AU)',     │
│  'link': 'https://www.apple.com/au/newsroom/2025/01/apple-reports-first-quarter-results/'}, {'title': 'Apple    │
│  (UK)', 'link': 'https://www.apple.com/uk/newsroom/2025/01/apple-reports-first-quarter-results/'}, {'t...       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Writer                                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  The latest earnings report from Apple has generated significant interest among investors and analysts alike.   │
│  The company's revenue for 2025 was $416.16 billion, a 6.43% increase compared to the previous year's $391.04   │
│  billion. Earnings were $112.01 billion, an increase of 19.50%.                                                 │
│                                                                                                                 │
│  The average rating for AAPL stock is "Buy," according to 29 analysts, with a 12-month stock price target of    │
│  $275.87, which represents a 1.27% upside from the latest price.                                                │
│                                                                                                                 │
│  Apple's App Store fees have been reduced for "mini apps" that integrate more of its software, from 30% to      │
│  15%. This move is seen as an effort to promote smaller developers and increase user engagement.                │
│                                                                                                                 │
│  The company has also announced plans to develop support for Apple CarPlay in Tesla vehicles, which could       │
│  potentially impact the automotive industry.                                                                    │
│                                                                                                                 │
│  Overall, the earnings report suggests that Apple remains a strong player in the technology sector, with a      │
│  solid financial performance and a focus on innovation. However, as with any major company, there are always    │
│  potential risks and challenges ahead.                                                                          │
│                                                                                                                 │
│  Key Takeaways:                                                                                                 │
│                                                                                                                 │
│  *   Revenue for 2025 was $416.16 billion, a 6.43% increase compared to the previous year.                      │
│  *   Earnings were $112.01 billion, an increase of 19.50%.                                                      │
│  *   The average rating for AAPL stock is "Buy," with a 12-month stock price target of $275.87.                 │
│  *   Apple has reduced its App Store fees for "mini apps" that integrate more of its software.                  │
│  *   The company plans to develop support for Apple CarPlay in Tesla vehicles.                                  │
│                                                                                                                 │
│  Investment Implications:                                                                                       │
│                                                                                                                 │
│  *   Investors may view the earnings report as a positive sign for Apple's financial performance and            │
│  innovation efforts.                                                                                            │
│  *   However, there are always potential risks and chal

/Users/macbook/Desktop/crewai_agent_001/.venv/lib/python3.11/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cae0d701-c94c-47f9-8658-7654db1e1ce7                                                                     │
│  Agent: Report Writer                                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3dbc63d1-1d34-4115-a1fd-a5f90cd35494                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: The latest earnings report from Apple has generated significant interest among investors and     │
│  analysts alike. The company's revenue for 2025 was $416.16 billion, a 6.43% increase compared to the previous  │
│  year's $391.04 billion. Earnings were $112.01 billion, an increase of 19.50%.                                  │
│                                                                                                                 │
│  The average rating for AAPL stock is "Buy," according to 29 analysts, with a 12-month stock price target of    │
│  $275.87, which represents a 1.27% upside from the latest price.                                                │
│                                                                                                                 │
│  Apple's App Store fees have been reduced for "mini apps" that integrate more of its software, from 30% to      │
│  15%. This move is seen as an effort to promote smaller developers and increase user engagement.                │
│                                                                                                                 │
│  The company has also announced plans to develop support for Apple CarPlay in Tesla vehicles, which could       │
│  potentially impact the automotive industry.                                                                    │
│                                                                                                                 │
│  Overall, the earnings report suggests that Apple remains a strong player in the technology sector, with a      │
│  solid financial performance and a focus on innovation. However, as with any major company, there are always    │
│  potential risks and challenges ahead.                                                                          │
│                                                                                                                 │
│  Key Takeaways:                                                                                                 │
│                                                                                                                 │
│  *   Revenue for 2025 was $416.16 billion, a 6.43% increase compared to the previous year.                      │
│  *   Earnings were $112.01 billion, an increase of 19.50%.                                                      │
│  *   The average rating for AAPL stock is "Buy," with a 12-month stock price target of $275.87.                 │
│  *   Apple has reduced its App Store fees for "mini apps" that integrate more of its software.                  │
│  *   The company plans to develop support for Apple CarPlay in Tesla vehicles.                                  │
│                                                                                                                 │
│  Investment Implications:                                                                                       │
│                                                                                                                 │
│  *   Investors may view the earnings report as a positive sign for Apple's financial performance and            │
│  innovation efforts.                                  

In [96]:
from IPython.display import Markdown

Markdown(result.raw)


The latest earnings report from Apple has generated significant interest among investors and analysts alike. The company's revenue for 2025 was $416.16 billion, a 6.43% increase compared to the previous year's $391.04 billion. Earnings were $112.01 billion, an increase of 19.50%.

The average rating for AAPL stock is "Buy," according to 29 analysts, with a 12-month stock price target of $275.87, which represents a 1.27% upside from the latest price.

Apple's App Store fees have been reduced for "mini apps" that integrate more of its software, from 30% to 15%. This move is seen as an effort to promote smaller developers and increase user engagement.

The company has also announced plans to develop support for Apple CarPlay in Tesla vehicles, which could potentially impact the automotive industry.

Overall, the earnings report suggests that Apple remains a strong player in the technology sector, with a solid financial performance and a focus on innovation. However, as with any major company, there are always potential risks and challenges ahead.

Key Takeaways:

*   Revenue for 2025 was $416.16 billion, a 6.43% increase compared to the previous year.
*   Earnings were $112.01 billion, an increase of 19.50%.
*   The average rating for AAPL stock is "Buy," with a 12-month stock price target of $275.87.
*   Apple has reduced its App Store fees for "mini apps" that integrate more of its software.
*   The company plans to develop support for Apple CarPlay in Tesla vehicles.

Investment Implications:

*   Investors may view the earnings report as a positive sign for Apple's financial performance and innovation efforts.
*   However, there are always potential risks and challenges ahead, such as increased competition in the technology sector.
*   Investors should consider these factors when making investment decisions.

Conclusion:

The latest earnings report from Apple has generated significant interest among investors and analysts alike. By considering various trading strategies and taking into account the company's solid financial performance and focus on innovation, investors can make informed decisions about their investments.